In [36]:
import os
import time
from meidic_vtach_utils.run_on_recommended_cuda import get_cuda_environ_vars as get_vars
os.environ.update(get_vars(select="* -3 -4"))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda.amp as amp

from IPython.display import display
import nibabel as nib
from torch.utils.checkpoint import checkpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from mdl_seg_class.metrics import dice3d
from mdl_seg_class.visualization import get_overlay_grid

import glob
import re
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

print(torch.__version__)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))


### Recommended gpus on this machine (descending order) ###
  ID  Card name              Util    Mem free  Cuda             User(s)
----  -------------------  ------  ----------  ---------------  -----------------------
   0  Tesla T4                0 %   15106 MiB  11.1(455.45.01)  hansen, graf
   1  Tesla T4                0 %   13633 MiB  11.1(455.45.01)  weihsbach, hansen, graf
   2  GeForce RTX 2080 Ti     0 %   11016 MiB  11.1(455.45.01)  hansen, graf
   4  GeForce RTX 2080 Ti     0 %   11016 MiB  11.1(455.45.01)  hansen, graf
   5  GeForce RTX 2080 Ti     0 %   11016 MiB  11.1(455.45.01)  hansen, graf
   3  Quadro RTX 8000      ! 90 %   28928 MiB  11.1(455.45.01)  hansen, graf
   6  GeForce RTX 2080 Ti  ! 94 %    2829 MiB  11.1(455.45.01)  hansen, graf, bigalke

Will apply following mapping

  ID  Card name        torch
----  -----------  --  -------
   0  Tesla T4     ->  cuda:0
1.9.1+cu102
7605
Tesla T4


In [15]:
class ASPPConv(nn.Sequential):
    def __init__(self, in_channels, out_channels, dilation):
        modules = [
            nn.Conv3d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU()
        ]
        super(ASPPConv, self).__init__(*modules)


class ASPPPooling(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super(ASPPPooling, self).__init__(
            # nn.AdaptiveAvgPool2d(1),
            nn.Conv3d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU())

    def forward(self, x):
        size = x.shape[-3:]
        # x = F.adaptive_avg_pool3d(x, (1))
        for mod in self:
            x = mod(x)
        return F.interpolate(x, size=size, mode='nearest')  # , align_corners=False)


class ASPP(nn.Module):
    def __init__(self, in_channels, atrous_rates, out_channels=256):
        super(ASPP, self).__init__()
        modules = [nn.Sequential(
            nn.Conv3d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU())]

        rates = tuple(atrous_rates)
        for rate in rates:
            modules.append(ASPPConv(in_channels, out_channels, rate))

        modules.append(ASPPPooling(in_channels, out_channels))

        self.convs = nn.ModuleList(modules)
 
        self.project = nn.Sequential(
            nn.Conv3d(len(self.convs) * out_channels, out_channels, 1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(),
            nn.Dropout(0.5))

    def forward(self, x):
        res = []
        for conv in self.convs:
            res.append(conv(x))
        res = torch.cat(res, dim=1)
        return self.project(res)


# Mobile-Net with depth-separable convolutions and residual connections
class ResBlock(torch.nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, inputs):
        return self.module(inputs) + inputs


def create_model(output_classes: int = 14,input_channels: int = 1):
    # in_channels = torch.Tensor([1,16,24,24,32,32,32,64]).long()
    in_channels = torch.Tensor([input_channels, 24, 24, 32, 48, 48, 48, 64]).long()
    mid_channels = torch.Tensor([64, 128, 192, 192, 256, 256, 256, 384]).long()
    out_channels = torch.Tensor([24, 24, 32, 48, 48, 48, 64, 64]).long()
    mid_stride = torch.Tensor([1, 1, 1, 2, 1, 1, 1, 1])
    net = [nn.Identity()]
    for i in range(8):
        inc = int(in_channels[i])
        midc = int(mid_channels[i])
        outc = int(out_channels[i])
        strd = int(mid_stride[i])
        layer = nn.Sequential(nn.Conv3d(inc, midc, 1, bias=False), nn.BatchNorm3d(midc), nn.ReLU6(True),
                              nn.Conv3d(midc, midc, 3, stride=strd, padding=1, bias=False, groups=midc),
                              nn.BatchNorm3d(midc), nn.ReLU6(True),
                              nn.Conv3d(midc, outc, 1, bias=False), nn.BatchNorm3d(outc))
        if i == 0:
            layer[0] = nn.Conv3d(inc, midc, 3, padding=1, stride=2, bias=False)
        if (inc == outc) & (strd == 1):
            net.append(ResBlock(layer))
        else:
            net.append(layer)

    backbone = nn.Sequential(*net)

    count = 0
    # weight initialization
    for m in backbone.modules():
        if isinstance(m, nn.Conv3d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out')
            count += 1
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, (nn.BatchNorm3d, nn.GroupNorm)):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.zeros_(m.bias)

    print('#CNN layer', count)
    # complete model: MobileNet + ASPP + head (with a single skip connection)
    # newer model (one more stride, no groups in head)
    aspp = ASPP(64, (2, 4, 8, 16, 32), 128)
    head = nn.Sequential(nn.Conv3d(128 + 24, 64, 1, padding=0, groups=1, bias=False), nn.BatchNorm3d(64), nn.ReLU(), \
                         nn.Conv3d(64, 64, 3, groups=1, padding=1, bias=False), nn.BatchNorm3d(64), nn.ReLU(), \
                         nn.Conv3d(64, output_classes, 1))
    return backbone, aspp, head


def apply_model(backbone, aspp, head, img, checkpointing=True, return_intermediate=False):
    if checkpointing:
        x1 = checkpoint(backbone[:3], img)
        x2 = checkpoint(backbone[3:], x1)
        y = checkpoint(aspp, x2)
        y1 = torch.cat((x1[0:1], F.interpolate(y[0:1], scale_factor=2)), 1)
        output_j = checkpoint(head, y1)
    else:
        x1 = backbone[:3](img)
        x2 = backbone[3:](x1)
        y = aspp(x2)
        y1 = torch.cat((x1[0:1], F.interpolate(y[0:1], scale_factor=2)), 1)
        output_j = head(y1)
    if return_intermediate:
        return y1,output_j
    else:
        return output_j


In [6]:
class CrossMoDa_Data(Dataset):
    def __init__(self,
        base_dir, domain, state,
        use_additional_data=False, resample = True,
        size:tuple = (96,96,60), normalize:bool = True):
        """
        Function to create Dataset structure with crossMoDa data.
        The function allows to use different preproccessing steps of the crossMoDa data set
        and using additinal data from TCIA database.
        The data can also be resampled to a desired size and normalized to mean=0 and std=1.

        Parameters:
                base_dir (os.Pathlike): provide the directory which contains "L1..." to "L4..." directories
                domain (str): choose which domain to load. Can be set to "source", "target" or "validation". Source are ceT1, target and validation hrT2 images.

                state (str): state of preprocessing:    "l1" = original data,
                                                        "l2" = resampled data @ 0.5mm,
                                                        "l3" = center-cropped data,
                                                        "l4" = image specific crops for desired anatomy

                use_additional_data (bool): set to True to use additional data from TCIA (default: False)

                resample (bool): set to False to disable resampling to desired size (default: True)

                size (tuple): 3d-tuple(int) to which the data is resampled. Unused if resample=False. (default: (96,96,60)).
                    WARNING: choosing large sizes or not resampling can lead to excess memory usage

                normalize (bool): set to False to disable normalization to mean=0, std=1 for each image (default: True)

        Returns:
                torch.utils.data.Dataset containing CrossMoDa data

        Useful Links:
        CrossMoDa challenge:
        https://crossmoda.grand-challenge.org/

        ToDos:
            extend to other preprocessing states

        Example:
            dataset = CrossMoDa_source('original')

            data = dataset.get_data()

        """

        #define finished preprocessing states here with subpath and default size
        states = {
            'l1':('L1_original/', (512,512,160)),
            'l2':('L2_resampled_05mm/', (420,420,360)),
            'l3':('L3_coarse_fixed_crop/', (128,128,192)),
            'l4':('L4_fine_localized_crop/', (128,128,128))
        }
        t0 = time.time()
        #choose directory with data according to chosen preprocessing state
        if state not in states: raise Exception("Unknown state. Choose one of: "+str(states.keys))
        state_dir = states[state.lower()][0] #get sub directory
        if not resample: size = states[state.lower()][1] #set size to default defined at top of file
        path = base_dir + state_dir
        #get file list
        if domain.lower() =="ceT1" or domain.lower() =="source":
            directory = "source_training_labeled/"
            add_directory = "__additional_data_source_domain__"
            domain = "ceT1"
        elif domain.lower() =="hrT2" or domain.lower() =="target":
            directory = "target_training_unlabeled/"
            add_directory = "__additional_data_target_domain__"
            domain = "hrT2"
        elif domain.lower() =="validation":
            directory = "target_validation_unlabeled/"
        else:
            raise Exception("Unknown domain. Choose either 'source', 'target' or 'validation'")
        files = sorted(glob.glob(os.path.join(path+directory , "*.nii.gz")))

        if domain == "hrT2":
            files = files+sorted(glob.glob(os.path.join(path+"__omitted_labels_target_training__" , "*.nii.gz")))
        if domain.lower() =="validation":
            files = files+sorted(glob.glob(os.path.join(path+"__omitted_labels_target_validation__" , "*.nii.gz")))
        if use_additional_data and domain.lower() != "validation": #add additional data to file list
            files = files+sorted(glob.glob(os.path.join(path+add_directory , "*.nii.gz")))
            files = [i for i in files if "additionalLabel" not in i] #remove additional label files

        #initialize variables
        self.imgs = torch.zeros(0,size[0],size[1],size[2])
        self.labels = torch.zeros(0,size[0],size[1],size[2])
        self.img_nums = []
        self.label_nums = []
        #load data
        print("Loading CrossMoDa {} images and labels...".format(domain))

        for i,f in enumerate(tqdm(files)):
            # tqdm.write(f"Loading {f}")
            if "Label" in f:
                self.label_nums.append(int(re.findall(r'\d+', os.path.basename(f))[0]))
                tmp = torch.from_numpy(nib.load(f).get_fdata())
                if resample: #resample image to specified size
                    tmp = F.interpolate(tmp.unsqueeze(0).unsqueeze(0), size=size,mode='nearest').squeeze()
                if tmp.shape != size: #for size missmatch use symmetric padding with 0
                    difs = [size[0]-tmp.size(0),size[1]-tmp.size(1),size[2]-tmp.size(2)]
                    pad = (difs[-1]//2,difs[-1]-difs[-1]//2,difs[-2]//2,difs[-2]-difs[-2]//2,difs[-3]//2,difs[-3]-difs[-3]//2)
                    tmp = F.pad(tmp,pad)
                self.labels = torch.cat((self.labels,tmp.unsqueeze(0)),dim=0)
            elif domain in f:
                self.img_nums.append(int(re.findall(r'\d+', os.path.basename(f))[0]))
                tmp = torch.from_numpy(nib.load(f).get_fdata())
                if resample: #resample image to specified size
                    tmp = F.interpolate(tmp.unsqueeze(0).unsqueeze(0), size=size,mode='trilinear',align_corners=False).squeeze()
                if normalize: #normalize image to zero mean and unit std
                    tmp = (tmp - tmp.mean()) / tmp.std()
                if tmp.shape != size: #for size missmatch use symmetric padding with 0
                    difs = [size[0]-tmp.size(0),size[1]-tmp.size(1),size[2]-tmp.size(2)]
                    pad = (difs[-1]//2,difs[-1]-difs[-1]//2,difs[-2]//2,difs[-2]-difs[-2]//2,difs[-3]//2,difs[-3]-difs[-3]//2)
                    tmp = F.pad(tmp,pad)
                self.imgs = torch.cat((self.imgs,tmp.unsqueeze(0)),dim=0)
        self.labels = self.labels.long()
        #check for consistency
        print("Equal image and label numbers: {}".format(self.img_nums==self.label_nums))
        print("Image shape: {}, mean.: {:.2f}, std.: {:.2f}".format(self.imgs.shape,self.imgs.mean(),self.imgs.std()))
        print("Label shape: {}, max.: {}".format(self.labels.shape,torch.max(self.labels)))
        print("Data import finished. Elapsed time: {:.1f} s".format(time.time()-t0 ))

    def __len__(self):
        return int(self.imgs.size(0))

    def __getitem__(self, idx):
        image = self.imgs[idx]
        label = self.labels[idx]
        return image, label.long()

    def get_data(self):
        return self.imgs,self.labels

    def get_image_numbers(self):
        return self.img_nums

    def get_label_numbers(self):
        return self.label_nums

In [29]:
training_dataset = CrossMoDa_Data("/share/data_supergrover1/weihsbach/shared_data/tmp/CrossMoDa/", 
domain="source", state="l4")
validation_dataset = CrossMoDa_Data("/share/data_supergrover1/weihsbach/shared_data/tmp/CrossMoDa/", 
domain="validation", state="l4")
target_dataset = CrossMoDa_Data("/share/data_supergrover1/weihsbach/shared_data/tmp/CrossMoDa/", 
domain="target", state="l4")

Loading CrossMoDa ceT1 images and labels...


100%|██████████| 418/418 [02:28<00:00,  2.82it/s]


Equal image and label numbers: True
Image shape: torch.Size([209, 96, 96, 60]), mean.: 0.00, std.: 1.00
Label shape: torch.Size([209, 96, 96, 60]), max.: 2
Data import finished. Elapsed time: 150.1 s
Loading CrossMoDa validation images and labels...


100%|██████████| 82/82 [00:13<00:00,  6.22it/s]


Equal image and label numbers: False
Image shape: torch.Size([63, 96, 96, 60]), mean.: -0.00, std.: 1.00
Label shape: torch.Size([19, 96, 96, 60]), max.: 2
Data import finished. Elapsed time: 13.3 s
Loading CrossMoDa hrT2 images and labels...


100%|██████████| 120/120 [00:21<00:00,  5.56it/s]


Equal image and label numbers: True
Image shape: torch.Size([60, 96, 96, 60]), mean.: 0.00, std.: 1.00
Label shape: torch.Size([60, 96, 96, 60]), max.: 2
Data import finished. Elapsed time: 21.9 s


In [10]:
def augmentAffine(img_in, seg_in, strength=0.05):
    """
    3D affine augmentation on image and segmentation mini-batch on GPU.
    (affine transf. is centered: trilinear interpolation and zero-padding used for sampling)
    :input: img_in batch (torch.cuda.FloatTensor), seg_in batch (torch.cuda.LongTensor)
    :return: augmented BxCxTxHxW image batch (torch.cuda.FloatTensor), augmented BxTxHxW seg batch (torch.cuda.LongTensor)
    """
    B,C,D,H,W = img_in.size()
    affine_matrix = (torch.eye(3,4).unsqueeze(0) + torch.randn(B, 3, 4) * strength).to(img_in.device)

    meshgrid = F.affine_grid(affine_matrix,torch.Size((B,1,D,H,W)))

    img_out = F.grid_sample(img_in, meshgrid,padding_mode='border')
    seg_out = F.grid_sample(seg_in.float(), meshgrid, mode='nearest')

    return img_out, seg_out



def augmentNoise(img_in,strength=0.05):
    return img_in + strength*torch.randn_like(img_in)


    
def pad_center_plane(img,size):
    s0,s1,s2 = img.size()
    i0 = (size[0]-s0)//2
    i1 = (size[1]-s1)//2
    i2 = 0
    pd = (i2,size[2]-s2-i2,i1,size[1]-s1-i1,i0,size[0]-s0-i0)
    #print('pad',pd)
    img = F.pad(img, pd, "constant", 0)
    return img



def crop_center_plane(img, size):
    s0,s1,s2 = img.size()
    i0 = (s0-size[0])//2
    i1 = (s1-size[1])//2
    i2 = 0
    img = img[i0:i0+size[0],i1:i1+size[1],i2:i2+size[2]]
    return img

In [58]:
#training routine

def train_DL(training_dataset, target_dataset, validation_dataset,
    epochs=500, update_epx = 50, use_mind = True, debug=False):

    if use_mind:
        C =12
    else:
        C = 1
    all_segs = torch.cat([seg for _, seg in training_dataset])

    num_class = int(torch.max(all_segs).item()+1)
    class_weight = torch.sqrt(1.0/(torch.bincount(all_segs.long().view(-1)).float()))
    class_weight = class_weight/class_weight.mean()
    class_weight[0] = 0.15
    class_weight = class_weight.cuda()
    print('inv sqrt class_weight', class_weight)

    kf = KFold(n_splits=2)
    kf.get_n_splits(training_dataset)
    train_idx, val_idx = next(kf.split(training_dataset))
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_dataloader = DataLoader(training_dataset, batch_size=1, shuffle=False,sampler=train_subsampler)
    target_dataloader = DataLoader(target_dataset, batch_size=1,shuffle=False)
    val_dataloader = DataLoader(validation_dataset, batch_size=1, shuffle=False,sampler=val_subsampler)

    backbone, aspp, head = create_model(output_classes=num_class,input_channels=C)
    optimizer = torch.optim.Adam(list(backbone.parameters())+list(aspp.parameters())+list(head.parameters()),lr=0.001)

    criterion = nn.CrossEntropyLoss(class_weight)
    scaler = amp.GradScaler()
    backbone.cuda() 
    backbone.train()
    aspp.cuda() 
    aspp.train()
    head.cuda() 
    head.train()
    t0 = time.time()

    for epx in range(epochs):
        optimizer.zero_grad()
        img, seg = next(iter(train_dataloader))
        img, seg = img.unsqueeze(0).float().cuda(), seg.unsqueeze(0).float().cuda()
        
        if use_mind:
            # img = mindssc(img)
            pass

        img, seg = augmentAffine(img, seg, strength=0.1)
        img = augmentNoise(img, strength=0.02)
        
        if debug:
            color_map = {
                0: None, 
                1: (255,0,0), #ONEHOT id and RGB color
                2: (0,255,0)
            }
            img_slices = img[0:1].permute(0,1,4,2,3).squeeze().unsqueeze(1)
            seg_slices = seg[0:1].permute(0,1,4,2,3).squeeze().to(dtype=torch.int64)
            idx_dept_with_segs, *_ = torch.nonzero(seg_slices > 0, as_tuple=True)
            idx_dept_with_segs = idx_dept_with_segs.unique()

            img_slices = img_slices[idx_dept_with_segs]
            seg_slices = seg_slices[idx_dept_with_segs]
            
            pil_ov, _ = get_overlay_grid(
                img_slices, 
                torch.nn.functional.one_hot(seg_slices,3), 
                color_map, n_per_row=10, alpha=.5
            )
            display(pil_ov)
            
        interpolated_seg = F.interpolate(seg, scale_factor=0.5, mode='nearest').squeeze(0).long()

        img.requires_grad = True
        #img_mr.requires_grad = True
        with amp.autocast(enabled=True):
            output_j = apply_model(backbone,aspp,head,img,checkpointing=True)
            loss = criterion(output_j, interpolated_seg)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if epx%update_epx==update_epx-1 or epx == 0:
            dice = dice3d(
                torch.nn.functional.one_hot(output_j.argmax(1), 3),
                torch.nn.functional.one_hot(interpolated_seg, 3), one_hot_torch_style=True)
            print('epx',epx,round(time.time()-t0,2),'s','loss',round(loss.item(),6),'dice mean', round(dice.mean().item(),4),'dice',dice)
        
        if debug:
            break
#    stat_cuda('Visceral training')
    backbone.cpu()
    aspp.cpu() 
    head.cpu()

    return backbone,aspp,head

In [59]:
def save_model(backbone,aspp,head,name):
    torch.save(backbone.state_dict(), name + '_backbone.pth')
    torch.save(aspp.state_dict(), name + '_aspp.pth')
    torch.save(head.state_dict(), name + '_head.pth')
    return None

def load_model(name,output_classes,input_channels):
    backbone, aspp, head = create_model(output_classes=output_classes,input_channels=input_channels)
    backbone.load_state_dict(torch.load( name + '_backbone.pth'))
    aspp.load_state_dict(torch.load(name + '_aspp.pth'))
    head.load_state_dict(torch.load(name + '_head.pth'))
    return backbone,aspp,head

epochs = 2000
updates = 50
# imgs = torch.cat((imgs_train_source,imgs_train_target),dim=0)
# label = torch.cat((labels_train_source,labels_train_target),dim=0)
backbone,aspp,head = train_DL(training_dataset, target_dataset, validation_dataset, 
    epochs=epochs, update_epx=updates, use_mind=False, debug=False
)

save_model(backbone, aspp, head,'data/models')
#backbone,aspp,head=load_model('Models/half_res_adapted_model_target_training',3,12)

inv sqrt class_weight tensor([0.1500, 0.6111, 2.3484], device='cuda:0')
#CNN layer 24
epx 0 0.33 s loss 1.200961 dice mean 0.1176 dice tensor([[3.5275e-01, 0.0000e+00, 1.4750e-04]])
epx 49 10.95 s loss 0.169627 dice mean 0.3775 dice tensor([[0.9972, 0.0000, 0.1353]])
epx 99 21.8 s loss 0.054101 dice mean 0.4736 dice tensor([[0.9998, 0.0000, 0.4211]])
epx 149 32.93 s loss 0.031083 dice mean 0.4835 dice tensor([[0.9995, 0.0000, 0.4510]])
epx 199 43.97 s loss 0.019888 dice mean 0.4602 dice tensor([[0.9996, 0.0000, 0.3810]])
epx 249 54.97 s loss 0.029535 dice mean 0.4008 dice tensor([[0.9958, 0.0000, 0.2066]])
epx 299 65.93 s loss 0.020487 dice mean 0.4372 dice tensor([[0.9982, 0.0000, 0.3134]])
epx 349 76.8 s loss 0.012713 dice mean 0.4284 dice tensor([[0.9995, 0.0000, 0.2857]])
epx 399 87.66 s loss 0.007376 dice mean 0.4346 dice tensor([[0.9995, 0.0000, 0.3043]])
epx 449 98.52 s loss 0.016268 dice mean 0.6855 dice tensor([[0.9979, 0.8399, 0.2188]])
epx 499 109.38 s loss 0.01205 dice mean

In [ ]:
path = '/share/data_sam1/ckruse/image_data/CrossMoDa/target_training/'
label_path = '/share/data_supergrover1/weihsbach/tmp/crossmoda_full_set/'
plot = False
backbone.cuda() 
aspp.cuda() 
head.cuda() 
target_dices = torch.zeros(32)
source_dices = torch.zeros(32)
for i in range(32):
    ind = i+150
    nii_img = nib.load(path + 'crossmoda_'+ str(ind) + '_hrT2.nii.gz')
    nii_label = nib.load(label_path + 'crossmoda_'+ str(ind) + '_hrT2_Label.nii.gz')
    tmp = torch.from_numpy(nii_img.get_fdata())
    label = torch.from_numpy(nii_label.get_fdata()).cuda()
    org_size = tmp.size()  
    size = (192,192,64)
    tmp = crop_center_plane(tmp,size)
    tmp = (tmp - tmp.mean()) / tmp.std()
    org_img = torch.from_numpy(nii_img.get_fdata())
    
    img= tmp.float().cuda()
    img = mindssc(img.unsqueeze(0).unsqueeze(0))
    with torch.no_grad():
        with amp.autocast(enabled=True):
            output_j = apply_model(backbone,aspp,head,img,checkpointing=False)#
    modeled_seg = F.interpolate(output_j,scale_factor=2).argmax(1)
    modeled_seg = pad_center_plane(modeled_seg.squeeze(),org_size)
    #print(modeled_seg.shape,torch.max(modeled_seg))
    connectivity = 18 # only 4,8 (2D) and 26, 18, and 6 (3D) are allowed
    np_label = modeled_seg.long().cpu().numpy().astype('int32')
    tmp = pad_center_plane(tmp.squeeze(),org_size)
    #plot = True
    if plot:
        slc = 30
        i0 = overlaySegment(tmp[:,:,slc].cpu(), modeled_seg[:,:,slc].cpu(), flag=False)
        i1 = overlaySegment(org_img[:,:,slc].cpu(), modeled_seg[:,:,slc].cpu(), flag=False)
        fig,axs = plt.subplots(1, 2,figsize=(18, 9))
        axs[0].imshow((i0+i1).cpu().numpy())
        axs[1].imshow(i1.cpu().numpy())
        plt.show()
        fig.set_figwidth(40)
        fig.set_figheight(10)
    #print(modeled_seg.shape,torch.max(modeled_seg))
    target_dices[i] = dice_coeff(modeled_seg,label)
    print(f'image: crossmoda_{ind}_hrT2.nii.gz, dice: {target_dices[i]*100:0.2f}')
    
print(f'target dice mean: {target_dices.mean()*100:0.2f}')


In [ ]:
path = '/share/data_sam1/ckruse/image_data/CrossMoDa/target_validation/'
plot = False
save = True
backbone.cuda() 
aspp.cuda() 
head.cuda() 
for i in range(32):
    ind = i+211
    nii_img = nib.load(path + 'crossmoda_'+ str(ind) + '_hrT2.nii.gz')
    img_affine = nii_img.affine
    tmp = torch.from_numpy(nii_img.get_fdata())
    org_img = torch.from_numpy(nii_img.get_fdata())
    org_size = tmp.size()  
    size = (192,192,64)
    tmp = crop_center_plane(tmp,size)
    #print('red. shape',tmp.shape)
    #tmp = F.interpolate(tmp.unsqueeze(0).unsqueeze(0), size=size)
    tmp = (tmp - tmp.mean()) / tmp.std()
        
    img= tmp.float().cuda()
    #print(img.shape)
    img = mindssc(img.unsqueeze(0).unsqueeze(0))
    with torch.no_grad():
        with amp.autocast(enabled=True):
            output_j = apply_model(backbone,aspp,head,img,checkpointing=False)#
    modeled_seg = F.interpolate(output_j,scale_factor=2).argmax(1)
    modeled_seg = pad_center_plane(modeled_seg.squeeze(),org_size)
    tmp = pad_center_plane(tmp.squeeze(),org_size)
    #print('org shape',tmp.shape)
    if plot:
        slc = 30
        i0 = overlaySegment(tmp[:,:,slc].cpu(), modeled_seg[:,:,slc].cpu(), flag=False)
        i1 = overlaySegment(org_img[:,:,slc].cpu(), modeled_seg[:,:,slc].cpu(), flag=False)
        fig,axs = plt.subplots(1, 2,figsize=(18, 9))
        axs[0].imshow((i0+i1).cpu().numpy())
        axs[1].imshow(i1.cpu().numpy())
        plt.show()
        fig.set_figwidth(40)
        fig.set_figheight(10)
    if save:
        label_nii = nib.Nifti1Image(modeled_seg.float().squeeze().cpu().numpy(), img_affine)
        nib.save(label_nii, 'Deeplab_validation/crossmoda_'+ str(ind) + '_Label.nii.gz')  
        

In [ ]:
#!rm Deeplab_validation.zip
#!zip -r Deeplab_validation_half_res_adapted_model_target_train.zip Deeplab_validation